In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import tensorflow as tf 
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Embedding, Activation, BatchNormalization, Dropout, Dot, Add, Flatten

from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping

from wordcloud import WordCloud

%matplotlib inline




In [2]:
!pip install matplotlib wordcloud


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 381.0 kB/s eta 0:00:20
   ------ --------------------------------- 1.3/8.1 MB 1.0 MB/s eta 0:00:07
   --------- ------------------------------ 1.8/8.1 MB 1.4 MB/s eta 0:00:05
   ---------- ----------------------------- 2.1/8.1 MB 1.3 MB/s eta 0:00:05
   -------------- ------------------------- 2.9/8.1 MB 1.6 MB/s eta 0:00:04
   --------------- ------------------------ 3.1/8.1 MB 1.6 MB/s eta 0:00:04
   ------------------ --------------------- 3.7/8.1 MB 1.6 M


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### READING ANIME LIST.csv

In [4]:
import os

In [5]:
INPUT_DIR = os.path.join("..", "artifacts","raw")



In [6]:
rating_df = pd.read_csv(INPUT_DIR + "/animelist.csv", low_memory=True, usecols=["user_id", "anime_id", "rating"])


In [7]:
rating_df.head()


,user_id,anime_id,rating
0,0,67,9
1,0,6702,7
2,0,242,10
3,0,4898,0
4,0,21,10


In [8]:
len(rating_df)




5000000

###Data Preprocessing


In [11]:
n_ratings = rating_df["user_id"].value_counts()
rating_df = rating_df[rating_df["user_id"].isin(n_ratings[n_ratings >= 400].index)].copy()






In [12]:
len(rating_df)

3246641

In [21]:
min_rating = min(rating_df["rating"])
max_rating = max(rating_df["rating"])







In [19]:
avg_rating = rating_df["rating"].mean()


In [23]:
max_rating


1.0

In [18]:
# min max scaler

rating_df["rating"] = rating_df["rating"].apply(lambda X: (X - min_rating) / (max_rating - min_rating)).values.astype(np.float64)


In [24]:
rating_df.isnull().sum()



user_id     0
anime_id    0
rating      0
dtype: int64

In [25]:
user_ids = rating_df['user_id'].unique().tolist()


In [ ]:
#### USER ---> 10054 ----> 98


In [ ]:
n_ratings = rating_df["anime_id"].value_counts()
rating_df = rating_df[rating_df["anime_id"].isin(n_ratings[n_ratings >= 100].index)]